In [70]:
import os
import docx
import re
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn import metrics
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer as snow



In [71]:
# Загрузка модели 
def get_model(path_model):
    with open(path_model, 'rb') as file:
        pickle_model = pickle.load(file)
    return pickle_model


In [72]:
# Поставить местоположение у тебя
path_model = r"C:\Users\79219\Desktop\digital_xakaton\model\LinearSVC_model.pkl"
model = get_model(path_model)

FileNotFoundError: [Errno 2] No such file or directory: 'model\\LinearSVC_model.pkl'